# Computing centralities from OSM data


Install and update `cityseer` if necessary.


In [ ]:
# !pip install --upgrade cityseer

## Data and Extents

This example will make use of OSM data downloaded from the [OSM API](https://wiki.openstreetmap.org/wiki/API) for a 5000m radius around London Soho.

For additional information on network preparation, see the corresponding discussion on [graph cleaning](https://cityseer.benchmarkurbanism.com/guide#graph-cleaning).


In [ ]:
from shapely import geometry

from cityseer.tools import graphs, io

# download from OSM
lng, lat = -0.13396079424572427, 51.51371088849723
buffer = 5000
# creates a WGS shapely polygon
poly_wgs, poly_utm, _utm_zone_number, _utm_zone_letter = io.buffered_point_poly(lng, lat, buffer)

## Load

We can now load the OSM dataset and compare three strategies:

1. Algorithmically cleaning the network
2. Using a messy uncleaned network
3. Using a messy network but with corrections through edge "dissolving" and "jitter".


In [ ]:
# generate OSM graph from polygon
G_utm = io.osm_graph_from_poly(poly_wgs, simplify=True, remove_parallel=True, iron_edges=False)
# decompose for higher resolution analysis
G_decomp = graphs.nx_decompose(G_utm, 25)
# prepare the data structures
nodes_gdf, _edges_gdf, network_structure = io.network_structure_from_nx(G_decomp, crs=32629)
# generate OSM graph from polygon
G_utm_messy = io.osm_graph_from_poly(poly_wgs, simplify=False)
G_utm_messy = graphs.nx_simple_geoms(G_utm_messy)
G_utm_messy = graphs.nx_remove_filler_nodes(G_utm_messy)
G_utm_messy = graphs.nx_remove_dangling_nodes(G_utm_messy, despine=15)
# decompose for higher resolution analysis
G_decomp_messy = graphs.nx_decompose(G_utm_messy, 25)
nodes_gdf_messy, _edges_gdf_messy, network_structure_messy = io.network_structure_from_nx(G_decomp_messy, crs=32629)
# generate dissolved weights
G_dissolved_wts = graphs.nx_weight_by_dissolved_edges(G_decomp_messy, 20)
nodes_gdf_dissolved, _edges_gdf_dissolved, network_structure_dissolved = io.network_structure_from_nx(
    G_dissolved_wts, crs=32629
)

## Calculate centralities

The centrality methods can now be computed.


In [ ]:
from cityseer.metrics import networks

distances = [250, 500, 2000]
# each of the below invocations will take around 5-6 minutes each depending on the available resources
# if computing wider area centralities, e.g. 20km, then use less decomposition to speed up the computation
nodes_gdf = networks.node_centrality_shortest(
    network_structure=network_structure,
    nodes_gdf=nodes_gdf,
    distances=distances,
)
# messy version
nodes_gdf_messy = networks.node_centrality_shortest(
    network_structure=network_structure_messy,
    nodes_gdf=nodes_gdf_messy,
    distances=distances,
)
# messy version with dissolved edge weights and shortest path jitter
nodes_gdf_dissolved = networks.node_centrality_shortest(
    network_structure=network_structure_dissolved,
    nodes_gdf=nodes_gdf_dissolved,
    distances=distances,
    jitter_scale=20,
)

## Plots

Let's plot a selection of distance thresholds for each of the computed measures to compare the different approaches.


In [ ]:
import matplotlib.pyplot as plt
from cityseer import rustalgos
from cityseer.tools import plot

bg_colour = "#111"
distances = [250, 500, 2000]
betas = rustalgos.betas_from_distances(distances)
avg_dists = rustalgos.avg_distances_for_betas(betas)
plot_bbox = poly_utm.centroid.buffer(1500).bounds
font_size = 7
for d, b, avg_d in zip(distances, betas, avg_dists):
    print(
        f"""
    "Gravity" index (spatial impedance weighted closeness-like centrality):
    Avg walking tolerance: {avg_d:.2f}m
    Beta: {b:.3f} (spatial impedance factor)
    Max walking tolerance: {d:.1f}m
    """
    )
    fig, axes = plt.subplots(1, 3, figsize=(8, 7), dpi=200, facecolor=bg_colour)
    plot.plot_scatter(
        axes[0],
        network_structure.node_xs,
        network_structure.node_ys,
        nodes_gdf[f"cc_metric_node_beta_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        face_colour=bg_colour,
    )
    axes[0].set_title("Algorithmically cleaned network", fontsize=font_size)
    plot.plot_scatter(
        axes[1],
        network_structure_messy.node_xs,
        network_structure_messy.node_ys,
        nodes_gdf_messy[f"cc_metric_node_beta_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        face_colour=bg_colour,
    )
    axes[1].set_title("Messy uncorrected network", fontsize=font_size)
    plot.plot_scatter(
        axes[2],
        network_structure_dissolved.node_xs,
        network_structure_dissolved.node_ys,
        nodes_gdf_dissolved[f"cc_metric_node_beta_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        face_colour=bg_colour,
    )
    axes[2].set_title("Messy w. dissolved weightings and jitter", fontsize=font_size)
    plt.show()

for d, b, avg_d in zip(distances, betas, avg_dists):
    print(
        f"""
    Spatial impedance weighted betweenness centrality:
    Avg walking tolerance: {avg_d:.2f}m
    Beta: {b:.3f} (spatial impedance factor)
    Max walking tolerance: {d:.1f}m
    """
    )
    fig, axes = plt.subplots(1, 3, figsize=(8, 7), dpi=200, facecolor=bg_colour)
    plot.plot_scatter(
        axes[0],
        network_structure.node_xs,
        network_structure.node_ys,
        nodes_gdf[f"cc_metric_node_betweenness_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        s_max=2,
        face_colour=bg_colour,
    )
    axes[0].set_title("Algorithmically cleaned network", fontsize=font_size)
    plot.plot_scatter(
        axes[1],
        network_structure_messy.node_xs,
        network_structure_messy.node_ys,
        nodes_gdf_messy[f"cc_metric_node_betweenness_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        s_max=2,
        face_colour=bg_colour,
    )
    axes[1].set_title("Messy uncorrected network", fontsize=font_size)
    plot.plot_scatter(
        axes[2],
        network_structure_dissolved.node_xs,
        network_structure_dissolved.node_ys,
        nodes_gdf_dissolved[f"cc_metric_node_betweenness_{d}"],
        bbox_extents=plot_bbox,
        cmap_key="magma",
        s_max=2,
        face_colour=bg_colour,
    )
    axes[2].set_title("Messy w. dissolved weightings and jitter", fontsize=font_size)
    plt.show()